[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/agentilt/Airbnb_EDA/blob/main/GROUP_1_AIRBNB.ipynb)


# INSTRUCTIONS

**Perform a basic exploratory data analysis of the Inside Airbnb dataset and answer some questions**

To complete the assignment, paste the answers where appropriate, and upload the full notebook at the end.


**Submission format**: One Jupyter notebook (`Airbnb_GroupX.ipynb`).

Please do *not* submit:
* A zip file
* A link to Google CoLab
* A file with the wrong extension
* A Python script


To complete the assignment, follow the steps. The data is attached below.


The grading criteria are, in decreasing order of importance and increasing object of subjectivity, as follows:


* Code has no errors, the whole notebook runs from top to bottom without modifications (35 %)
* Code gives correct answers (30 %)
* Code avoids repetition and favours pandas methods where appropriate (loops and conditionals only if strictly necessary) (15 %)
* Code uses meaningful, explanatory variable names (10 %)
Code is as succinct as possible (when there are two ways of doing something, the simplest, shortest, or easier to understand is chosen) (5 %)
  * If you discuss several ways of doing something, with its pros and cons (without just dumping the code and no explanations), that counts positively as well
* Code is easy to read (i.e. "similar to how the professor codes") (5 %)


Optionally, you can include code comments describing the intent (i.e. code comments should answer "why is this code here?", not "what is this code doing?") and supplementary markdown cells if appropriate.



All the questions can be done independently of each other (after reading the data). They are not sorted in any particular order of difficulty.

#IMPORT LIBRARIES AND DATASET


In [ ]:

import pandas as pd
import numpy as np
df = pd.read_csv('listings.csv')


# PART 1: BASIC EXPLORATORY ANALYSIS

**Question 1**

How many rows does the dataset have? (Excluding the header containing the column names)

In [5]:
# Use len() function to count the number of rows in the DataFrame
# len(df) returns the number of rows (excluding the header)
# f"..." is an f-string (formatted string) - allows us to insert variables using {}
# The variable len(df) is evaluated and inserted into the string
print(f"Number of rows: {len(df)}")


Number of rows: 25000


**Question 2**

How many columns does the dataset have? (Excluding the autogenerated numerical index)

In [6]:
# df.columns returns a list-like object containing all column names
# len(df.columns) counts how many columns are in the DataFrame
# f-string formats the output with the column count inserted
print(f"Number of columns: {len(df.columns)}")


Number of columns: 79


**Question 3**

How many unique values are there for host_id?

In [7]:
# Access the 'host_id' column using df['host_id'] 
# .nunique() counts the number of unique (distinct) values in the column
# This tells us how many different hosts there are 
# Store the result in variable 'unique_hosts' for later use
unique_hosts = df['host_id'].nunique()

# Print the result using f-string formatting
# {unique_hosts} inserts the value of the variable into the string
print(f"Number of unique host_id values: {unique_hosts}")

Number of unique host_id values: 10453


**Question 4**

Count how many listings are there per host (where 1 row = 1 listing). Find the host with the largest number of listings. How many listings do they have?

In [8]:
# .value_counts() counts how many times each unique value appears in the column
# Returns a Series with counts for each host_id
listings_per_host = df['host_id'].value_counts()

# .idxmax() returns the index (host_id) with the maximum count value
# .max() returns the maximum count value itself
# We use both in the f-string to show which host has the most listings and how many
# Method chaining: we call .idxmax() and .max() directly on the Series
print(f"Host with the largest number of listings (host_id: {listings_per_host.idxmax()}) has {listings_per_host.max()} listings")

Host with the largest number of listings (host_id: 438136382) has 407 listings


**Question 5**

How many distinct hosts are superhosts?

In [9]:
# This uses boolean indexing 
# df['host_is_superhost'] == 't' creates a boolean Series (True/False for each row)
# df[...] filters rows where the condition is True (only superhosts)
# ['host_id'] selects just the host_id column from filtered rows
# .nunique() counts unique host_ids 
# Note: 't' means 'true' in this dataset (Airbnb uses 't'/'f' for boolean values)
superhosts = df[df['host_is_superhost'] == 't']['host_id'].nunique()

# Print the count of distinct superhosts
print(f"Number of distinct superhosts: {superhosts}")


Number of distinct superhosts: 2543


**Question 6**

In the city of Madrid there are 2 administrative levels represented in the dataset: neighbourhood and district. Find the district with the largest number of listings. How many does it have?

In [10]:
# Count how many listings each district has
# 'neighbourhood_group_cleansed' contains the district names
# .value_counts() returns a Series with district names as index and counts as values
listings_per_district = df['neighbourhood_group_cleansed'].value_counts()

# Find the district with maximum listings
# .idxmax() gets the district name (index) with highest count
# .max() gets the actual maximum count value
print(f"District with the largest number of listings: {listings_per_district.idxmax()} with {listings_per_district.max()} listings")

District with the largest number of listings: Centro with 10230 listings


**Question 7**

What's the average price of listings? (Error of +-1 USD is accepted)

In [11]:
# Price column is stored as text (string) with '$' and commas
# We need to convert it to a number to calculate the average
# Method chaining: multiple operations in sequence
# .str.replace('$', '') removes dollar signs from all values
# .str.replace(',', '') removes commas (thousands separators)
# .astype(float) converts the string to a float (decimal number)
# We create a new column 'price_numeric' to store the converted values
df['price_numeric'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

# Calculate the mean (average) of all prices
# .mean() is a pandas method that calculates the average of numeric values
average_price = df['price_numeric'].mean()

# Print with formatting: ${average_price:.2f}
# :.2f means format as float with 2 decimal places
print(f"Average price: ${average_price:.2f}")


Average price: $156.69


**Question 8**

How many listings have zero reviews?

In [12]:
# Boolean comparison: df['number_of_reviews'] == 0 creates True/False for each row
# Parentheses () group the operation
# .sum() on a boolean Series counts True values (True=1, False=0 when summed)
# This counts how many rows have number_of_reviews equal to 0
zero_reviews = (df['number_of_reviews'] == 0).sum()

# Print the count of listings with zero reviews
print(f"Number of listings with zero reviews: {zero_reviews}")

Number of listings with zero reviews: 5147


**Question 9**

Fill the gap: "Listings that are instantly bookable have an average number of reviews per month X % higher than those that are not" (Error of +-1 % is accepted)

In [13]:
# Filter rows where instant_bookable == 't' (true), then get reviews_per_month column, then calculate mean
# This gives us the average reviews per month for instantly bookable listings
instant_bookable_avg = df[df['instant_bookable'] == 't']['reviews_per_month'].mean()

# Same process but for listings that are not instantly bookable 
not_instant_bookable_avg = df[df['instant_bookable'] == 'f']['reviews_per_month'].mean()

# Calculate percentage difference: ((new - old) / old) * 100
# This is the standard formula for percentage increase
# Ternary operator: if not_instant_bookable_avg != 0 then calculate, else return 0
# This prevents division by zero error if there are no non-instant-bookable listings
percentage_difference = ((instant_bookable_avg - not_instant_bookable_avg) / not_instant_bookable_avg) * 100 if not_instant_bookable_avg != 0 else 0

# Print all three values with appropriate formatting
# :.2f formats to 2 decimal places, :.1f formats to 1 decimal place
print(f"Average reviews per month (instant bookable): {instant_bookable_avg:.2f}")
print(f"Average reviews per month (not instant bookable): {not_instant_bookable_avg:.2f}")
print(f"Percentage difference: {percentage_difference:.1f}%")

Average reviews per month (instant bookable): 2.09
Average reviews per month (not instant bookable): 1.50
Percentage difference: 39.7%


**Question 10**

How many listings have missing (null) license information?

In [14]:
# .isna() checks each value in the column and returns True if it's NaN (Not a Number/missing)
# Returns a boolean Series (True for missing, False for present)
# .sum() counts the True values (missing values)
# This tells us how many listings have null/missing license information
missing_license = df['license'].isna().sum()

# Print the count of missing license values
print(f"Number of listings with missing license information: {missing_license}")


Number of listings with missing license information: 15812


**Question 11**

Some licenses have a very long string starting with ES, followed by 2 letters (type of listing), followed by 2 letters (category), followed by a long list of numbers and some extra characters.

How many listings have a license containing the string "ESFC"?

In [15]:
# .str.contains() searches for a substring within each string value
# 'ESFC' is the pattern we're looking for
# na=False means: if a value is NaN (missing), return False instead of NaN
# Returns a boolean Series (True if 'ESFC' found, False otherwise)
# .sum() counts how many True values (how many contain 'ESFC')
esfc_license_count = df['license'].str.contains('ESFC', na=False).sum()

# Print the count of licenses containing 'ESFC'
print(f"Number of listings with license containing 'ESFC': {esfc_license_count}")


Number of listings with license containing 'ESFC': 4790


**Question 12**

How many listings have declared "Exempt" or "En proceso" in the license field?

In [16]:
# .str.contains() with regex pattern 'Exempt|En proceso'
# The | symbol means "OR" in regular expressions 
# So this searches for either "Exempt" OR "En proceso" in the license field
# case=False means case-insensitive search (matches "EXEMPT", "exempt", "Exempt", etc.)
# na=False handles missing values by returning False
# .sum() counts how many licenses contain either string
exempt_or_proceso = df['license'].str.contains('Exempt|En proceso', case=False, na=False).sum()

# Print the count
print(f"Number of listings with 'Exempt' or 'En proceso' in license: {exempt_or_proceso}")


Number of listings with 'Exempt' or 'En proceso' in license: 964


**Question 13**

How many hosts *cannot* be contacted by email?

In [ ]:
# We want to find hosts who CANNOT be contacted by email
# ~ is the NOT operator in pandas 
# df['host_verifications'].str.contains('email', na=False) finds rows WITH email
# ~ inverts it to find rows WITHOUT email
# [~...] filters to only rows where email is NOT found
# ['host_id'] selects the host_id column from filtered rows
# .nunique() counts unique host_ids (distinct hosts without email)
hosts_without_email = df[~df['host_verifications'].str.contains('email', na=False)]['host_id'].nunique()

# Print the count of hosts without email contact
print(f"Number of hosts that cannot be contacted by email: {hosts_without_email}")


Number of hosts that cannot be contacted by email: 1344


**Question 14**

What's the maximum number of amenities found in any listing?

In [ ]:
# Amenities are stored as a JSON-like string
# To count amenities, we count commas and add 1 (since n items = n-1 commas)
# .fillna('[]') replaces missing values with empty array string '[]' before processing
# This prevents errors when working with NaN values
# .str.count(',') counts commas in each string
# + 1 because if there are 2 commas, there are 3 items
df['amenities_count'] = df['amenities'].fillna('[]').str.count(',') + 1

# Fix empty arrays: if amenities is '[]' (empty), set count to 0
# df.loc[...] selects specific rows and columns for assignment
# Condition: df['amenities'].fillna('[]').str.strip() == '[]' finds empty arrays
# .str.strip() removes whitespace, == '[]' checks if it's an empty array
# 'amenities_count' is the column we're updating
# = 0 sets the count to 0 for empty arrays
df.loc[df['amenities'].fillna('[]').str.strip() == '[]', 'amenities_count'] = 0

# Find the maximum value in the amenities_count column
# .max() returns the highest count value
max_amenities = df['amenities_count'].max()

# Print the maximum number of amenities found
print(f"Maximum number of amenities in any listing: {max_amenities}")

Maximum number of amenities in any listing: 95


**Question 15**

Which year has the record for number of hosts registered?

In [21]:
# Extract year from host_since column 
# pd.to_datetime() converts string dates to datetime objects (Python date format)
# .dt.year extracts just the year part (2010, 2011, etc.) from each date
# Creates a new column 'host_since_year' with just the year
df['host_since_year'] = pd.to_datetime(df['host_since']).dt.year

# Group data by year and count unique hosts registered in each year
# .groupby('host_since_year') groups rows by year
# ['host_id'] selects the host_id column from each group
# .nunique() counts unique host_ids in each year (distinct hosts registered)
# Result: a Series with years as index and host counts as values
hosts_per_year = df.groupby('host_since_year')['host_id'].nunique()

# Find which year has the maximum number of hosts
# .idxmax() returns the index (year) with the maximum value
year_with_max_hosts = hosts_per_year.idxmax()

# Get the actual maximum count value
max_hosts_count = hosts_per_year.max()

# Print the year and count
print(f"Year with the record for number of hosts registered: {year_with_max_hosts} ({max_hosts_count} hosts)")

Year with the record for number of hosts registered: 2016.0 (1287 hosts)


# PART 2: OPEN ENDED ANALYSIS

**Question 16**

Examine the "license" field a bit more closely. There are different structures present, apart from the one described above. Try to identify them and count how many listings have each type of license.

In [22]:
# Explore the license field to identify different license structures
# Check for ESFC pattern using regular expression 
# r'^ES[A-Z]{2}[A-Z]{2}' means: starts with ES, then 2 uppercase letters, then 2 uppercase letters
# ^ means "starts with", [A-Z]{2} means exactly 2 uppercase letters
# regex=True enables regex pattern matching
# Returns boolean Series (True if pattern matches)
esfc_pattern = df['license'].str.contains(r'^ES[A-Z]{2}[A-Z]{2}', na=False, regex=True)

# Check for "Exempt" text (case-insensitive)
exempt = df['license'].str.contains('Exempt', case=False, na=False)

# Check for "En proceso" text (case-insensitive)
en_proceso = df['license'].str.contains('En proceso', case=False, na=False)

# Build a pandas Series to store license type counts
# pd.Series() creates a Series 
# Dictionary format: {'label': value}
# .sum() on boolean Series counts True values (how many match each pattern)
license_types = pd.Series({
    'ESFC pattern (ES + 2 letters + 2 letters + numbers)': esfc_pattern.sum(),
    'Exempt': exempt.sum(),
    'En proceso': en_proceso.sum(),
    'Missing/Null': df['license'].isna().sum()
})

# Find licenses that don't match any of the standard patterns
# ~ negates each boolean Series (NOT esfc, NOT exempt, NOT en_proceso)
# & combines conditions with AND (all must be True)
# df['license'].notna() ensures we only look at non-missing values
# Result: boolean mask identifying non-standard licenses
non_standard_mask = ~esfc_pattern & ~exempt & ~en_proceso & df['license'].notna()

# Check if there are any non-standard licenses
# .any() returns True if any value in the Series is True
if non_standard_mask.any():
    # Filter to non-standard licenses and count occurrences
    # df.loc[mask, 'license'] selects license column where mask is True
    # .value_counts() counts how many times each unique license appears
    # .head(10) gets only the top 10 most common non-standard licenses
    other_licenses = df.loc[non_standard_mask, 'license'].value_counts().head(10)
    
    # Modify the index (license names) to add "Other: " prefix
    # .index accesses the index (row labels) of the Series
    # .str[:50] truncates long license names to first 50 characters
    other_licenses.index = 'Other: ' + other_licenses.index.str[:50]
    
    # Combine the standard license types with non-standard ones
    # pd.concat() concatenates/combines multiple Series into one
    # [license_types, other_licenses] is a list of Series to combine
    license_types = pd.concat([license_types, other_licenses])
    print("Other license patterns found:")

# Display all license type counts
# .to_string() formats the Series as a readable string table
print("License type counts:")
print(license_types.to_string())


Other license patterns found:
License type counts:
ESFC pattern (ES + 2 letters + 2 letters + numbers)     6221
Exempt                                                   661
En proceso                                               303
Missing/Null                                           15812
Other: 350202309690                                       27
Other: VT REGISTRADA                                      23
Other: EXENTO - Alquiler de Temporada según la LAU        21
Other: VT                                                 21
Other: AM-336                                             20
Other: 106/2006/1387                                      18
Other: VT-3483                                            16
Other: VT-14732                                           12
Other: VT-5980                                            10
Other: 350/2022/13778                                     10


**Question 17**

The "host_location" information is somewhat structured. Sometimes it contains (city, country), sometimes it doesn't. Explore how many different countries are present, trying to pay attention to typos, special values (like state names), and which are the most prevalent ones, other than Spain.

In [23]:
# Extract country from host_location field
# Format is usually "City, Country" (e.g., "Madrid, Spain")
# Method chaining: multiple string operations in sequence
# .fillna('') replaces missing values with empty string (prevents errors)
# .str.split(',') splits each string at commas, returns list ['City', ' Country']
# .str[-1] gets the last element of each list (the country part)
# .str.strip() removes leading/trailing whitespace
# Creates new column 'extracted_country' with just country names
df['extracted_country'] = df['host_location'].fillna('').str.split(',').str[-1].str.strip()

# Count how many hosts are from each country
# .value_counts() counts occurrences of each unique country
country_counts = df['extracted_country'].value_counts()

# Filter to show top countries excluding Spain and empty strings
# country_counts.index accesses the country names (index of the Series)
# != 'Spain' excludes Spain, != '' excludes empty strings
# & combines both conditions (both must be True)
# .head(20) gets the top 20 countries by count
top_countries = country_counts[(country_counts.index != 'Spain') & (country_counts.index != '')].head(20)

# Display the top countries
print("Top countries (excluding Spain):")
# .to_string() formats as a readable table
print(top_countries.to_string())

# Analyze data quality: check for typos and variations
# .nunique() counts unique values (helps identify if there are typos/variations)
# \n creates a new line in the output
print(f"\nTotal unique country values (including variations): {df['extracted_country'].nunique()}")
print(f"Total unique host_location values: {df['host_location'].nunique()}")


Top countries (excluding Spain):
extracted_country
United Kingdom          115
France                   58
Germany                  44
Mexico                   43
Argentina                40
Colombia                 38
FL                       24
Italy                    23
CA                       23
Brazil                   23
NY                       22
Switzerland              18
Netherlands              14
Peru                     13
Portugal                 12
United Arab Emirates     10
Venezuela                10
Australia                 8
Panama                    8
United States             7

Total unique country values (including variations): 77
Total unique host_location values: 440


**Question 18**

A few listings seem to be extremely expensive. Devise a method of extracting price outliers, and inspect those listings. Which ones have been most reviewed? Do they have more amenities than average? Highlight anything else that's interesting about them.

In [24]:
# Use IQR (Interquartile Range) method to identify price outliers
# This is a statistical method to find unusually high or low values
# Q1 (first quartile): 25% of prices are below this value
# .quantile(0.25) finds the 25th percentile
Q1 = df['price_numeric'].quantile(0.25)

# Q3 (third quartile): 75% of prices are below this value
# .quantile(0.75) finds the 75th percentile
Q3 = df['price_numeric'].quantile(0.75)

# IQR = Interquartile Range = difference between Q3 and Q1
# This measures the spread of the middle 50% of data
IQR = Q3 - Q1

# Calculate boundaries for outliers
# Lower bound: Q1 - 1.5*IQR (for extremely cheap listings)
lower_bound = Q1 - 1.5 * IQR

# Upper bound: Q3 + 1.5*IQR (for extremely expensive listings)
# Values above this are considered outliers
upper_bound = Q3 + 1.5 * IQR

# Filter DataFrame to only include outliers (prices above upper_bound)
# df['price_numeric'] > upper_bound creates boolean mask
# df[...] filters rows where condition is True
# .copy() creates a copy (good practice to avoid modifying original data)
outliers = df[df['price_numeric'] > upper_bound].copy()

# Display basic outlier statistics
# len(outliers) counts number of rows (outlier listings)
print(f"Number of price outliers (above ${upper_bound:.2f}): {len(outliers)}")
print(f"\nOutlier price statistics:")
# Calculate min, max, and mean prices for outliers
# .min(), .max(), .mean() are pandas aggregation methods
print(f"  Min price: ${outliers['price_numeric'].min():.2f}")
print(f"  Max price: ${outliers['price_numeric'].max():.2f}")
print(f"  Mean price: ${outliers['price_numeric'].mean():.2f}")

# Find the most reviewed outlier listings
# .nlargest(5, 'number_of_reviews') gets 5 rows with highest review counts
# [['id', 'name', ...]] selects specific columns to display
# This creates a smaller DataFrame with just the columns we want
most_reviewed_outliers = outliers.nlargest(5, 'number_of_reviews')[['id', 'name', 'price_numeric', 'number_of_reviews', 'amenities_count', 'host_is_superhost', 'review_scores_rating']]
print(f"\nTop 5 most reviewed outliers:")
# .to_string() formats the DataFrame as a readable table
print(most_reviewed_outliers.to_string())

# Compare amenities between outliers and all listings
# Calculate average amenities for outliers
avg_amenities_outliers = outliers['amenities_count'].mean()

# Calculate average amenities for all listings
avg_amenities_all = df['amenities_count'].mean()

# Display comparison
print(f"\nAverage amenities:")
print(f"  Outliers: {avg_amenities_outliers:.2f}")
print(f"  All listings: {avg_amenities_all:.2f}")
print(f"  Difference: {avg_amenities_outliers - avg_amenities_all:.2f}")

# Calculate additional insights about outliers
print(f"\nOther insights about outliers:")
# Calculate percentage of outliers that are superhosts
# (outliers['host_is_superhost'] == 't').sum() counts superhosts
# len(outliers) is total number of outliers
# * 100 converts to percentage
# :.1f formats to 1 decimal place
print(f"  Superhost percentage: {(outliers['host_is_superhost'] == 't').sum() / len(outliers) * 100:.1f}%")

# Calculate average review score for outliers
print(f"  Average review score: {outliers['review_scores_rating'].mean():.2f}")

# Calculate average number of reviews for outliers
print(f"  Average number of reviews: {outliers['number_of_reviews'].mean():.1f}")


Number of price outliers (above $305.00): 1183

Outlier price statistics:
  Min price: $306.00
  Max price: $25654.00
  Mean price: $788.97

Top 5 most reviewed outliers:
            id                                                name  price_numeric  number_of_reviews  amenities_count host_is_superhost  review_scores_rating
363    3146468                   Cozy room in the center of Madrid          627.0                603               14                 f                  4.82
1292  11990713  1-12.Las Cortes.Sol.Madrid Center.120m2 Bright.A.C          430.0                574               36                 t                  4.84
829    7571127  Calle Mayor, central, bright & spacious, AC, WiFi.          536.0                568               37                 t                  4.67
695    6542642      The Best Location-2 Steps From Gran Vía! (v10)          312.0                546               48                 f                  4.88
4395  30761904  Stunning Centrally Loca